## .:: Lab 03 ::.

This homework is a kaggle-competition.
Link to join the competition: https://www.kaggle.com/t/1aeb25700aa34f3cbd8aad23c43ea5c6
Please make sure to use your real name so that your participation could be evaluated.

How to submit your solutions: see code in the notebook.
- Before  the final deadline you have to send a notebook with your best solution to `cosmic.research.ml@yandex.ru`
- Name your file according to this convention: `2021_lab03_GroupNumber_Surname_Name.ipynb`
- Attach your .ipynb to an email with topic `2021_lab03_GroupNumber_Surname_Name.ipynb`
- Send it to `cosmic.research.ml@yandex.ru`
- Final Deadline is `2021-11-03 23:00:00 +03:00`

# -----------------------------------------------------------------------------------------

Dota 2 is a computer game in the MOBA (Multiplayer Online Battle Arena) genre. 
It is played by two teams, called **Radiant** and **Dire** which consist of five players each. 
The main goal of the game is to destroy other team's “Ancient", located at the opposite corners of the map. 
Matches are generated from a queue, taking into account the level of the game all players, known as MMR (Match Making Rank).

Each player earns gold and experience during the game, gets items and performs kills. 
Information about each match includes statistics of each player and of the whole team.

Your goal is to predict an outcome of the match: **whether radiant win or lose**

**File descriptions:**
* train_db.csv - the training set
* test_db.csv - the test set
* hero_stats.json - supplemental information about heroes that are used by the players
* lab03_baseline.ipynb - a sample notebook that includes sample solution
* baseline_submission.csv - a sample submission

**Train and Test Data fields:**
* match_id - an identifier of a match
* radiant_win - outcome of the match (train set only)
* hero - hero chosen by player (goes with prefix r1-r5 for radiant and d1-d5 for dire)
* level - level of a player
* xp - xp gained by the player
* gold - amount of gold acquired by the player
* lh - number of neutral creatures killed by the player
* kills - number of kills performed by the player
* deaths - times the player died
* items - number of items obtained by the player

Also there are some columns that reflect team statistics: first_blood, courier_time, etc.

# ------------------------------------------------------------------------------

Data can be downloaded from here: https://disk.yandex.ru/d/7aiYfUHOzX3IYw

In [1]:
#train_db = pd.read_csv('/kaggle/input/cosmic-research-ml-2021-lab03/train_db.csv', sep=',')
#test_db = pd.read_csv('/kaggle/input/cosmic-research-ml-2021-lab03/test_db.csv', sep=',')
#hero_stats = pd.read_json('/kaggle/input/cosmic-research-ml-2021-lab03/hero_stats.json')

In [2]:
#train_db = pd.read_csv("train_db.csv")
#test_db = pd.read_csv("test_db.csv")

In [3]:
#train_db.shape, test_db.shape, hero_stats.shape

In [4]:
#train_db.head()

In [5]:
#Y_train = train_db["radiant_win"]
#X_train = train_db.drop(columns=["radiant_win"])

In [6]:
#X_train.shape, Y_train.shape

In [7]:
#X_train = X_train.fillna(0)
#X_test = test_db.fillna(0)

In [8]:
#X_train.shape, X_test.shape, Y_train.shape

In [9]:
#from sklearn.tree import DecisionTreeClassifier

#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import roc_auc_score

# Public Baseline

In [10]:
#clf = DecisionTreeClassifier()
#cvs = cross_val_score(clf, 
#                      X_train, Y_train, 
#                      cv=5, 
#                      scoring='roc_auc', 
#                      n_jobs=-1)
#cvs_mean = np.mean(cvs)
#print(cvs)

In [11]:
#clf.fit(X_train, Y_train)

In [12]:
#predictions = clf.predict_proba(X_test)[:, 1]

In [13]:
#submit_data = test_db[["match_id"]].copy()
#submit_data.loc[:, "radiant_win"] = predictions
#submit_data.to_csv("public_baseline_submission.csv", index=None)

In [14]:
#submit_data

# My solve

In [15]:
import numpy as np
import pandas as pd

In [16]:
train_db = pd.read_csv('/kaggle/input/cosmic-research-ml-2021-lab03/train_db.csv', sep=',')
test_db = pd.read_csv('/kaggle/input/cosmic-research-ml-2021-lab03/test_db.csv', sep=',')
hero_stats = pd.read_json('/kaggle/input/cosmic-research-ml-2021-lab03/hero_stats.json')

In [17]:
train_db.shape, test_db.shape, hero_stats.shape

In [18]:
1 - (train_db.shape[0] - train_db.dropna().shape[0]) / train_db.shape[0]

In [19]:
1 - (test_db.shape[0] - test_db.dropna().shape[0]) / test_db.shape[0]

Странно, что в тестовых данных есть **nan**

In [20]:
print(set([type(train_db[i][0]) for i in train_db.columns]))

Все фичи и результаты - числа

Заменим все **nan** на **0**

In [21]:
new_train_db = train_db.fillna(0)
new_test_db = test_db.fillna(0)

In [22]:
Y_train = new_train_db["radiant_win"]
X_train = new_train_db.drop(columns=["radiant_win"])

X_test = new_test_db

In [23]:
Y_train.shape[0], X_train.shape[0], X_test.shape[0]

# feature importances

In [24]:
from time import time
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf = RandomForestClassifier()

start = time()
rf.fit(X_train[:10000], Y_train[:10000])
end = time()

print("fit {:.3f} seconds".format(end - start))

In [26]:
rf_coefs = rf.feature_importances_  
ncoef = rf_coefs.shape[0]
default_x = np.arange(ncoef)

In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
X = default_x
Y = rf_coefs
fig.add_trace(go.Bar(x = X, y = Y, name='RandomForestClassifier'))
fig.show()

In [28]:
ind = X[Y>0.01]
ind

In [29]:
head = list(X_train.columns.values) 
name_ind = [head[i] for i in ind]
name_ind

In [30]:
X_train_smaller = X_train[name_ind]
X_test_smaller  = X_test[name_ind]

# train - val - test

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
RANDOM_SEED = 42

In [33]:
Z_train, Z_not_train, F_train, F_not_train = train_test_split(X_train_smaller, 
                                                              Y_train,
                                                              test_size= 3/7, 
                                                              shuffle=True, 
                                                              stratify = None,
                                                              random_state = RANDOM_SEED)

Z_test, Z_val, F_test, F_val = train_test_split(Z_not_train, 
                                                F_not_train, 
                                                test_size= 1/3,
                                                shuffle=True, 
                                                stratify= None,
                                                random_state = RANDOM_SEED)

# FIT

https://tproger.ru/translations/scikit-learn-in-python/

In [34]:
from sklearn.neighbors    import KNeighborsClassifier
from sklearn              import svm
from sklearn.tree         import DecisionTreeClassifier
from sklearn.ensemble     import RandomForestClassifier


from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.metrics       import mean_squared_error

# 1. k-nearest neighbors

In [35]:
KNN = KNeighborsClassifier()

In [36]:
grid = {"n_neighbors": np.arange(2,21),                                     # default=5
        #"weights":     np.array(['uniform', 'distance']),                   # default=’uniform’
        #"algorithm":   np.array(['auto', 'ball_tree', 'kd_tree', 'brute']), # default=’auto’
        "metric":      np.array(['euclidean', 'manhattan', 'chebyshev'])   }

In [37]:
start = time()
GSCV = GridSearchCV(KNN, 
                    param_grid = grid,
                    cv = 5, 
                    return_train_score = True,
                    n_jobs = -1)
GSCV.fit(Z_train[:7000], F_train[:7000])
end = time()
print('{:.3f} seconds'.format(end - start))
print('best params:{}'.format(GSCV.best_params_))  
#print('best estimator:', GSCV.best_estimator_)  
#print('best score: {:.3f}'.format(GSCV.best_score_))

best params:{'metric': 'manhattan', 'n_neighbors': 17}

In [38]:
#cvs = cross_val_score(GSCV, Z_val, F_val, cv=5, n_jobs=-1)
#mean_cvs = np.mean(cvs)
#print('cross validation score:', cvs)
#print('mean cross validation score:', mean_cvs)

In [39]:
KNN = KNeighborsClassifier(metric = GSCV.best_params_.get('metric'), 
                           n_neighbors = GSCV.best_params_.get('n_neighbors'))
KNN.fit(Z_train, F_train)
F_train_pred = KNN.predict(Z_train)
F_test_pred  = KNN.predict(Z_test)

In [40]:
KNN_train_mse = mean_squared_error(F_train, F_train_pred)
KNN_test_mse = mean_squared_error(F_test, F_test_pred)

KNN_train_score = KNN.score(Z_train, F_train)
KNN_test_score  = KNN.score(Z_test, F_test)

cvs = cross_val_score(KNN, Z_val, F_val, cv=5, n_jobs=-1)
KNN_mean_cvs = np.mean(cvs)

In [41]:
KNN_test_mse, KNN_test_score, KNN_mean_cvs

# 2. Support Vector Machines

https://scikit-learn.org/stable/modules/svm.html#svm-classification

**SVC, NuSVC and LinearSVC** are classes capable of performing binary and multi-class classification on a dataset.

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [42]:
SVC = svm.SVC(kernel = 'linear')

In [57]:
c = np.arange(0,100,10)
c[0] = 1
print(c)
grid = {#"kernel": ['linear', 'poly'], #, 'rbf', 'sigmoid', 'precomputed']}
        "C" :  c}

In [ ]:
start = time()
GSCV = GridSearchCV(SVC, 
                    param_grid = grid,
                    cv = 5, 
                    return_train_score = True,
                    n_jobs = -1)
GSCV.fit(Z_train[:100], F_train[:100])
end = time()
print('{:.3f} seconds'.format(end - start), 
      '\nbest params:{}'.format(GSCV.best_params_))

In [ ]:
SVC = svm.SVC(kernel = 'linear', C = 70) 
SVC.fit(Z_train, F_train)
F_train_pred = SVC.predict(Z_train)
F_test_pred  = SVC.predict(Z_test)

In [ ]:
SVC_train_mse = mean_squared_error(F_train, F_train_pred)
SVC_test_mse = mean_squared_error(F_test, F_test_pred)

SVC_train_score = SVC.score(Z_train, F_train)
SVC_test_score  = SVC.score(Z_test, F_test)

cvs = cross_val_score(SVC, Z_val, F_val, cv=5, n_jobs=-1)
SVC_mean_cvs = np.mean(cvs)

SVC_test_mse, SVC_test_score, SVC_mean_cvs

# 3. Decision Tree

In [ ]:
DT = DecisionTreeClassifier()

In [ ]:
grid = {"max_depth": np.arange(1,51),
        "criterion": np.array(['gini','entropy'])}

In [ ]:
start = time()
GSCV = GridSearchCV(DT, 
                    param_grid = grid ,
                    cv = 5, 
                    return_train_score = True,
                    n_jobs = -1)
GSCV.fit(Z_train, F_train)
end = time()
print('{:.3f} seconds'.format(end - start))
print('best params:{}'.format(GSCV.best_params_))  

In [ ]:
DT = DecisionTreeClassifier(criterion = GSCV.best_params_.get('criterion'), 
                            max_depth = GSCV.best_params_.get('max_depth'))
DT.fit(Z_train, F_train)
F_train_pred = DT.predict(Z_train)
F_test_pred  = DT.predict(Z_test)

In [ ]:
DT_train_mse  = mean_squared_error(F_train, F_train_pred)
DT_test_mse   = mean_squared_error(F_test, F_test_pred)

DT_train_score  = DT.score(Z_train, F_train)
DT_test_score   = DT.score(Z_test, F_test)

cvs = cross_val_score(DT,Z_val,F_val,cv=5,n_jobs=-1)
DT_mean_cvs = np.mean(cvs)

DT_test_mse, DT_test_score, DT_mean_cvs

# 4. Random Forest 

In [ ]:
RF = RandomForestClassifier()

In [ ]:
grid = {"n_estimators": np.arange(1, 200, 20),
        "max_features": np.arange(1, Z_train.shape[1]+1)}

In [ ]:
start = time()
GSCV = GridSearchCV(RF, 
                    param_grid = grid ,
                    cv = 5, 
                    return_train_score = True,
                    n_jobs = -1)
GSCV.fit(Z_train[:7000], F_train[:7000])
end = time()
print('{:.3f} seconds'.format(end - start))
print('best params:{}'.format(GSCV.best_params_))  

In [ ]:
RF = RandomForestRegressor(n_estimators = GSCV.best_params_.get('n_estimators'), 
                           max_features = GSCV.best_params_.get('max_features'))
RF.fit(Z_train, F_train)
F_train_pred = RF.predict(Z_train)
F_test_pred  = RF.predict(Z_test)

In [ ]:
RF_train_mse = mean_squared_error(F_train, F_train_pred)
RF_test_mse  = mean_squared_error(F_test, F_test_pred)

RF_train_score = RF.score(Z_train, F_train)
RF_test_score  = RF.score(Z_test, F_test)

cvs = cross_val_score(RF,Z_val,F_val,cv=5,n_jobs=-1)
RF_mean_cvs = np.mean(cvs)


DT_test_mse, DT_test_score, DT_mean_cvs

# 5. Naive Bayes

In [ ]:
NB = 

# 6. Linear Discriminant Analysis

In [ ]:
LDA = 

# 7. Logistic Regression

In [ ]:
LogR = LogisticRegression()

In [ ]:
p = ['l2', 'none'] 
#p = ['l1', 'l2', 'elasticnet', 'none']
#t = np.arange(1e-5,1e-3,1e-4)                    # default = 1e-4
c = np.logspace(-3, 3, endpoint=True, base = 10)  # default = 1.0

In [ ]:
grid = {#"penalty": p,
        #"tol"    :t,
        "C"      : c}

In [ ]:
start = time()
GSCV = GridSearchCV(LogR, 
                    param_grid = grid ,
                    cv = 5, 
                    return_train_score = True,
                    n_jobs = -1)
GSCV.fit(Z_train, F_train)
end = time()
print('{:.3f} seconds'.format(end - start))
print('best params:{}'.format(GSCV.best_params_))  
#print('best estimator:', GSCV.best_estimator_)  
#print('best score: {:.3f}'.format(GSCV.best_score_))

In [ ]:
#cvs = cross_val_score(GSCV, Z_val, F_val, cv=5, n_jobs=-1)
#mean_cvs = np.mean(cvs)
#print('cross validation score:', cvs)
#print('mean cross validation score:', mean_cvs)

In [ ]:
LogR = LogisticRegression(penalty = 'l2', #GSCV.best_params_.get('penalty'), 
                          C = GSCV.best_params_.get('C'))
LogR.fit(Z_train, F_train)
F_train_pred = LogR.predict(Z_train)
F_test_pred  = LogR.predict(Z_test)

In [ ]:
LogR_train_score = LogR.score(Z_train,F_train)
LogR_test_score  = LogR.score(Z_test,F_test)
#print('Score for trainSet: {:.3f}'.format(LogR_train_score))
#print('Score for testSet: {:.3f}'.format(LogR_test_score))

LogR_train_mse   = mean_squared_error(F_train, F_train_pred)
LogR_test_mse    = mean_squared_error(F_test, F_test_pred)
#print('Mean squared error for trainSet: {:.3f}'.format(LogR_train_mse ))
#print('Mean squared error for testSet: {:.3f}'.format(LogR_test_mse))

cvs = cross_val_score(LogR, Z_val, F_val, cv=5, n_jobs=-1)
LogR_mean_cvs = np.mean(cvs)
#print('cross validation score:', cvs)
#print('mean cross validation score:', LogR_mean_cvs)

# Analize

In [ ]:
model = ['LinR','LogR','KNN','DT','RF']
train_score = [LinR_train_score,LogR_train_score,KNN_train_score,DT_train_score,RF_train_score]
test_score  = [LinR_test_score,LogR_test_score,KNN_test_score,DT_test_score,RF_test_score]
train_mse = [LinR_train_mse,LogR_train_mse,KNN_train_mse,DT_train_mse,RF_train_mse]
test_mse  = [LinR_test_mse,LogR_test_mse,KNN_test_mse,DT_test_mse,RF_test_mse]
mean_cvs = [LinR_mean_cvs,LogR_mean_cvs,KNN_mean_cvs,DT_mean_cvs,RF_mean_cvs]

In [ ]:
dt = [model,train_score,test_score,train_mse,test_mse,mean_cvs]
dt = np.array(dt,dtype=object).T.tolist()

In [ ]:
pd.DataFrame(dt,columns=['cls','train_score','test_score','train_mse','test_mse','mean_cvs']) 
#=None, index=None, columns=None, dtype=None, copy=False)

Наименьшкю ошибку показывает RF. Возможно, потому что наиболее важные фичи были отобраны с помощью него. 

При этом, наибольший test_score у LogR

# ---------------------------------------------------------------------------------------

# 6. Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
#best params:{'max_features': 3, 'n_estimators': 141}
clf_a = RandomForestRegressor(n_estimators = 141, max_features = 3)
clf_a.fit(Z_train, F_train)

#best params:{'criterion': 'gini', 'max_depth': 12}
clf_b = DecisionTreeClassifier(criterion = 'gini', max_depth = 12)
clf_b.fit(Z_train, F_train)

In [ ]:
type(RandomForestRegressor)

In [ ]:
clf_c = VotingClassifier(estimators=[('rf', clf_a),
                                     ('dt', clf_b)], 
                         voting ='hard')

In [ ]:
w1 = list(np.linspace(0, 1, 11))
w2 = [1 - i for i in w1]
#print(w1,'\n',w2)
w = []
for i in range(len(w1)):
    w.append([w1[i],w2[i]])

In [ ]:
params = {'weights': w}

GSCV = GridSearchCV(estimator=clf_c, 
                    param_grid=params, 
                    cv=5, 
                    n_jobs=-1)

GSCV.fit(Z_train, F_train)
print(GSCV.best_params_)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.ensemble import VotingClassifier